## Install dependencies

In [6]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, '/research/harris/vivian/v1-models/V1-models/')
import V1_models
import robustbench

ModuleNotFoundError: No module named 'robustbench'

First let's enable GPU support in Colab (*Runtime -> Change runtime type -> Hardware accelerator -> GPU*). 

Now let's try to load CIFAR-10 and one of the most robust CIFAR-10 models from [Unlabeled Data Improves Adversarial Robustness](https://arxiv.org/abs/1905.13736) 
that achieves 59.53% robust accuracy evaluated with AutoAttack under Linf epsilon of 8/255 on 10k points:

In [3]:
from robustbench.data import load_cifar10
x_test, y_test = load_cifar10(n_examples=50)

from robustbench.utils import load_model
carmon = load_model(model_name='Carmon2019Unlabeled', threat_model='Linf')
engstrom = load_model(model_name='Engstrom2019Robustness', thread_model='Linf')
v1 = V1_models.BN_V1_V1_LinearLayer_CIFAR10(100, 2, 0.1, 1, True)
gaussian = control_models.Gaussian_CIFAR10(100, scale=1, bias=True, seed=None)
uniform = control_models.Uniform_CIFAR10(100, bias=True, seed=None)
v1.load_state_dict(torch.load("/research/harris/vivian/v1-models/saved-models/CIFAR10/BN_V1_V1_Linear_bias_true/trial_1/model.pt", map_location=device))
gaussian.load_state_dict(torch.load("/research/harris/vivian/v1-models/saved-models/CIFAR10/Gaussian_Control_bias_true/trial_1/model.pt", map_location=device))
uniform.load_state_dict(torch.load("/research/harris/vivian/v1-models/saved-models/CIFAR10/Uniform_Control_bias_true/trial_1/model.pt", map_location=device))

ModuleNotFoundError: No module named 'robustbench'

For other models available in the **Model Zoo**, see the model IDs in [this table](https://github.com/AdvBench/advbench#model-zoo-list-of-models).

## Evaluate your favourite attack from FoolBox

Let's try to evaluate the robustness of this model. One can use any favourite library for this. For example, [FoolBox](https://github.com/bethgelab/foolbox) is a well-established library that implements many different attacks. We can start from a simple PGD attack and evaluate it on 50 points to illustrate how it works:

In [6]:
#!pip install -q foolbox  # produces 2 incompatibility messages, but they can be just ignored
import foolbox as fb

model_fb = fb.PyTorchModel(model, bounds=(0, 1))
_, advs, success = fb.attacks.LinfPGD(rel_stepsize=0.1, steps=20)(model_fb, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])
print('Robust accuracy: {:.1%}'.format(1 - success.float().mean()))

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## AutoAttack evaluation

Wonderful! Can we do better with a more accurate attack?

Let's try to evaluate its robustness with a cheap version [AutoAttack](https://arxiv.org/abs/2003.01690) from 
ICML 2020 with 2/4 attacks (only APGD-CE and APGD-DLR) and again on 50 points:

In [ ]:
#!pip install -q git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack

adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary.apgd.n_restarts = 1
x_adv = adversary.run_standard_evaluation(x_test, y_test)

Note that for our standardized evaluation of Linf-robustness we use the *full* version of AutoAttack which is slower but 
more accurate (for that just use `adversary = AutoAttack(model, norm='Linf', eps=8/255)`).

## Common corruptions

What about other types of perturbations? Is Lp-robustness useful there? We can evaluate the available models on more general perturbations. For example, let's take images corrupted by *fog* perturbations from [CIFAR-10-C](https://github.com/hendrycks/robustness) with the highest level of severity (5). Are different Linf robust models perform better on them?

In [ ]:
from robustbench.data import load_cifar10c
from robustbench.utils import clean_accuracy

corruptions = ['fog']
x_test, y_test = load_cifar10c(n_examples=1000, corruptions=corruptions, severity=5)   

for model_name in ['Standard', 'Engstrom2019Robustness', 'Rice2020Overfitting', 'Carmon2019Unlabeled']:
  model = load_model(model_name)
  acc = clean_accuracy(model, x_test, y_test)
  print('Model: {}, CIFAR-10-C accuracy: {:.1%}'.format(model_name, acc))

As we can see, **all** these Linf robust models perform considerably worse than the standard model on this type of corruptions. 
This curious phenomenon was first noticed in [Adversarial Examples Are a Natural Consequence of Test Error in Noise](https://arxiv.org/abs/1901.10513) and explained from the frequency perspective in [A Fourier Perspective on Model Robustness in Computer Vision](https://arxiv.org/abs/1906.08988). 

However, on average adversarial training *does* help on CIFAR-10-C. One can check this easily by loading all types of corruptions via `load_cifar10c(n_examples=1000, severity=5)`, and repeating evaluation on them.

## Concluding remarks
We hope that having a *simple* and *unified* access to an up-to-date list of the most robust models will facilitate new insights about benefits and tradeoffs in robustness with respect to different types of perturbations.

Feel free to contribute a new interesting dataset or analysis with novel insights! We will be happy to include it in the repository.